<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Project Title: Paraphrase Generation Using T5-FineTuning</b></h1>



### Tabel of Contents:



* [Import Libraries](#1)

* [Load Dataset](#2)

* [T5 Model](#4)

* [Evaluation](#3)

* [Predictive for Test](#8)

* [Like this? Upvote and comment! 🌊 End](#6)

**<a id="1"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Import Libraries</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [2]:
!pip install transformers datasets wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from huggingface_hub import login
import wandb
wandb.init(mode = "disabled")
import warnings
warnings.filterwarnings('ignore')

**<a id="2"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Load Dataset</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [4]:
# Load the PAWS dataset
dataset = load_dataset("paws", "labeled_final")
dataset

README.md:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [5]:
def preprocess_pows(dataset, label=1):
  df = pd.DataFrame(dataset)
  df = df[df['label']==label]
  df['input_text'] = df['sentence1']
  df['target_text'] = df['sentence2']
  return df[['input_text','target_text']]

train_df = preprocess_pows(dataset['train'])
test_df = preprocess_pows(dataset['test'])
validation_df = preprocess_pows(dataset['validation'])
train_df.head()

,input_text,target_text
1,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...
3,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...
4,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...
5,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ..."
7,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...


In [6]:
test_df.head()

,input_text,target_text
2,"In January 2011 , the Deputy Secretary General...","In January 2011 , FIBA Asia deputy secretary g..."
5,"During her sophomore , junior and senior summe...","During her second , junior and senior summers ..."
7,"His father emigrated to Missouri in 1868 , but...",His father emigrated to Missouri in 1868 but r...
9,It is situated south of Köroğlu Mountains and ...,It is situated south of Köroğlu - mountains an...
10,The Río Blanco mine is a large copper mine loc...,The Río Blanco - Mine is a large copper mine i...


In [7]:
validation_df.head()

,input_text,target_text
1,They were there to enjoy us and they were ther...,They were there for us to enjoy and they were ...
2,"After the end of the war in June 1902 , Higgin...","In August , after the end of the war in June 1..."
3,From the merger of the Four Rivers Council and...,Shawnee Trails Council was formed from the mer...
4,The group toured extensively and became famous...,The group toured extensively and was famous in...
5,Kathy and her husband Pete Beale ( Peter Dean ...,Kathy and her husband Peter Dean ( Pete Beale ...


In [8]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(validation_df)
train_dataset

Dataset({
    features: ['input_text', 'target_text', '__index_level_0__'],
    num_rows: 21829
})

<a id="4"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">T5 Model</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [9]:
# Initialize tokenizer and model
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
max_length = max([len(tokenizer.encode(text)) for text in train_dataset['input_text']])
print(f"Calculated max_length: {max_length}")

Calculated max_length: 79


In [11]:
# Tokenization function
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], truncation=True, padding="max_length", max_length=max_length)
    targets = tokenizer(examples['target_text'], truncation=True, padding="max_length", max_length=max_length)
    inputs['labels'] = targets['input_ids']
    return inputs
# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
validation_dataset = validation_dataset.map(tokenize_function, batched=True)
print(train_dataset[0])

Map:   0%|          | 0/21829 [00:00<?, ? examples/s]

Map:   0%|          | 0/3536 [00:00<?, ? examples/s]

Map:   0%|          | 0/3539 [00:00<?, ? examples/s]

{'input_text': 'The NBA season of 1975 -- 76 was the 30th season of the National Basketball Association .', 'target_text': 'The 1975 -- 76 season of the National Basketball Association was the 30th season of the NBA .', '__index_level_0__': 1, 'input_ids': [37, 14512, 774, 13, 16312, 1636, 3, 3959, 47, 8, 604, 189, 774, 13, 8, 868, 21249, 2125, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [37, 16312, 1636, 3, 3959, 774, 13, 8, 868, 21249, 2125, 47, 8, 604, 189, 774, 13, 8, 14512, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [12]:
training_args = TrainingArguments(
    output_dir="./Finetuning_T55_Paraphrase_Generation",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset
)

In [13]:
trainer.train()
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.225400,0.193508


In [14]:
tokenizer.save_pretrained(training_args.output_dir)

('./Finetuning_T55_Paraphrase_Generation/tokenizer_config.json',
 './Finetuning_T55_Paraphrase_Generation/special_tokens_map.json',
 './Finetuning_T55_Paraphrase_Generation/spiece.model',
 './Finetuning_T55_Paraphrase_Generation/added_tokens.json')

In [ ]:
login(token="HUGGINGFACE_TOKEN")

In [16]:
repo_name = "ahmed792002/Finetuning_T55_Paraphrase_Generation"
trainer.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

events.out.tfevents.1733254537.b7a3d3c0f34b.213.0:   0%|          | 0.00/7.00k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/ahmed792002/Finetuning_T55_Paraphrase_Generation/commit/f753ba0f955684bc34f6601942985652f756dd4e', commit_message='Upload tokenizer', commit_description='', oid='f753ba0f955684bc34f6601942985652f756dd4e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ahmed792002/Finetuning_T55_Paraphrase_Generation', endpoint='https://huggingface.co', repo_type='model', repo_id='ahmed792002/Finetuning_T55_Paraphrase_Generation'), pr_revision=None, pr_num=None)

**<a id="1"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 10px 10px 10px #042b4c">Evaluation</p></b></h1>

<a class="btn" href="#home">Tabel of Contents</a>

In [17]:
results = trainer.evaluate(test_dataset)
print("Evaluation results:")
print("Test Loss",results["eval_loss"])

Evaluation results:
Test Loss 0.19574232399463654


<a id="8"></a>

<h1><b><p style="background-image: url(https://i.postimg.cc/0Qwf8YX6/2927262.jpg);font-family:camtasia;font-size:110%;color:white;text-align:center;border-radius:15px 50px; padding:7px; border:solid 2px #09375b; box-shadow: 20px 10px 10px #042b4c">Predictive for Test</p></b></h1>

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def generate_paraphrase(input_text, max_length=max_length, num_return_sequences=4, temperature=1.0):
    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_length, padding="max_length")
    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}
    # Generate paraphrases using beam search
    outputs = model.generate(
        **inputs,
        max_length=max_length + 20,
        num_return_sequences=num_return_sequences,
        num_beams=num_return_sequences,
        no_repeat_ngram_size=2,
        temperature=temperature
    )
    # Decode generated outputs
    paraphrased_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return paraphrased_texts

In [29]:
test_sentence = test_df["input_text"].iloc[50]
paraphrased_sentences = generate_paraphrase(test_sentence,num_return_sequences=5)
# Display results
print(f"Original: {test_sentence}")
for i, paraphrase in enumerate(paraphrased_sentences, 1):
    print(f"Paraphrase {i}: {paraphrase}")

Original: Both trains are hauled by a Guwahati Loco Shed based WDM-2 diesel locomotive from Rangiya to Dibrugarh and vice versa .
Paraphrase 1: Both trains are hauled by a Guwahati Loco Shed based WDM-2 diesel locomotive from Rangiya to Dibrugarh and vice versa .
Paraphrase 2: Both trains are hauled by a Guwahati Loco Shed based diesel locomotive WDM-2 from Rangiya to Dibrugarh and vice versa .
Paraphrase 3: Both trains are hauled by a Guwahati Loco Shed WDM-2 diesel locomotive from Rangiya to Dibrugarh and vice versa .
Paraphrase 4: Both trains are carried by a Guwahati Loco Shed based WDM-2 diesel locomotive from Rangiya to Dibrugarh and vice versa .
Paraphrase 5: Both trains are hauled by a Guwahati Loco Shed diesel locomotive WDM-2 from Rangiya to Dibrugarh and vice versa .


In [31]:
test_sentence = test_df["input_text"].iloc[200]
paraphrased_sentences = generate_paraphrase(test_sentence,num_return_sequences=10)
# Display results
print(f"Original: {test_sentence}")
for i, paraphrase in enumerate(paraphrased_sentences, 1):
    print(f"Paraphrase {i}: {paraphrase}")

Original: On 1 April 2010 Robertson made the first official maximum break of his career in his second round match in the 2010 China Open against Peter Ebdon .
Paraphrase 1: On 1 April 2010 Robertson made the first official maximum break of his career in his second round match against Peter Ebdon in the 2010 China Open .
Paraphrase 2: On April 1 , 2010 Robertson made the first official maximum break of his career in his second round match against Peter Ebdon in the 2010 China Open .
Paraphrase 3: On 1 April 2010 Robertson made the first official maximum break of his career in his second round match in the 2010 China Open against Peter Ebdon .
Paraphrase 4: On 1 April 2010 , Robertson made the first official maximum break of his career in his second round match against Peter Ebdon in the 2010 China Open .
Paraphrase 5: On 1 April 2010 Robertson made the first official maximum break of his career in his second round match against Peter Ebdon at the 2010 China Open .
Paraphrase 6: On April

In [32]:
test_sentence = test_df["input_text"].iloc[1000]
paraphrased_sentences = generate_paraphrase(test_sentence,num_return_sequences=3)
# Display results
print(f"Original: {test_sentence}")
for i, paraphrase in enumerate(paraphrased_sentences, 1):
    print(f"Paraphrase {i}: {paraphrase}")

Original: The 2009 Superstars Series season was the third season of Campionato Italiano Superstars ( Italian Superstars Championship ) and the sixth season of the International Superstars Series .
Paraphrase 1: The 2009 Superstars Series season was the third season of the Campionato Italiano SuperStars ( Italian Super Stars Championship ) and the sixth season in the international superstar series .
Paraphrase 2: The 2009 Superstars Series season was the third season of the Campionato Italiano SuperStars ( Italian Super Stars Championship ) and the sixth season in the International SuperSTAR Series .
Paraphrase 3: The 2009 Superstars Series season was the third season of the Campionato Italiano SuperStars ( Italian Super Stars Championship ) and the sixth season in the International SuperSTARs series .


In [33]:
test_sentence = test_df["input_text"].iloc[2000]
paraphrased_sentences = generate_paraphrase(test_sentence,num_return_sequences=15)
# Display results
print(f"Original: {test_sentence}")
for i, paraphrase in enumerate(paraphrased_sentences, 1):
    print(f"Paraphrase {i}: {paraphrase}")

Original: Rafic Hariri ( born 1980-1981 ) is a Lebanese billionaire , heir and developer , the youngest son of Fahd Hariri .
Paraphrase 1: Rafic Hariri ( born 1980-1981 ) is a Lebanese billionaire , heir and developer . He is the youngest son of Fahd 's father ;
Paraphrase 2: Rafic Hariri ( born 1980-1981 ) is a Lebanese billionaire , heir and developer . He is the youngest son of Fahd 's son ;
Paraphrase 3: Rafic Hariri ( born 1980-1981 ) is a Lebanese billionaire , heir and developer . He is the youngest son of Fahd 's father !
Paraphrase 4: Rafic Hariri ( born 1980-1981 ) is a Lebanese billionaire , heir and developer . He is the youngest son of Fahd 's son !
Paraphrase 5: Rafic Hariri ( born 1980-1981 ) is a Lebanese billionaire , heir and developer . He is the youngest son of Fahd 's son ...
Paraphrase 6: Rafic Hariri ( born 1980-1981 ) is a Lebanese billionaire , heir and developer . He is the youngest son of Fahd 's father ...
Paraphrase 7: Rafic Hariri ( born 1980-1981 ) is a L

<center><span style="font-family:Palatino; font-size:22px;"><i>Like this? <span style="color:#DC143C;">Upvote and Comment!</span> </i>🌊 End</span> </center>